# Import Modules

In [ ]:
import os
print(os.getcwd())
import sys

import pickle

import numpy as np
import pandas as pd

import chart_studio.plotly as py
import plotly.graph_objs as go

from ccf_similarity.ccf import CCF

from active_learning.al_analysis import ALPerformance

from plotting.my_plotly import my_plotly_plot

from layout import layout

In [ ]:
from inputs import files_list_gp_ucb, files_list_random

from inputs import stoich_i

# Script Inputs

In [ ]:
# perc_of_structs = 2.5
# num_disc = 7
# subdirs_list = ["gp_ucb", "random"]
# shared_scatter_props = dict(
#     mode="lines",
#     )

data_path_root = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/00_ml_workflow",
    "191102_new_workflow/00_abx_al_runs/out_data",
    stoich_i)


from inputs import top_ids_to_track_ab2, top_ids_to_track_ab3

In [ ]:
if stoich_i == "AB2":
    top_ids_to_track = top_ids_to_track_ab2
elif stoich_i == "AB3":
    top_ids_to_track = top_ids_to_track_ab3
else:
    print("ISDJIFSDJI")

In [ ]:
color_i = "rgba(0,100,255,0.5)"

In [ ]:
files_list=files_list_gp_ucb
subdir="gp_ucb_True/01_attempt"
unique_scatter_props=dict(
    name="gp_ucb w/ dupl",
    marker=dict(color=color_i),
    error_y=dict(
        color=color_i,
        ),
    )
# shared_scatter_props=shared_scatter_props
ALPerf_account_duplicates=True
top_ids_to_track=top_ids_to_track
# color_i = "rgba(0,100,255,0.5)"
color2="rgba(0,100,255,1.)"

In [ ]:
out_data_dict = dict()

# #############################################################################
if files_list is not None:
    tmp = 42
else:
    files_list = os.listdir(
        os.path.join(
            # dir_i,
            data_path_root,
            # "out_data",
            subdir))
    files_list = [i for i in files_list if "pickle" in i]
    files_list = [i for i in files_list if "AL_" in i]

# print(files_list)

data_dict = dict()
for file_i in files_list:
    # #########################################################################
    num = file_i.split("_")[-1].split(".")[0]

    file_path_i =os.path.join(
        data_path_root, subdir, file_i)
    with open(file_path_i, "rb") as fle:
        AL_i = pickle.load(fle)

    data_dict[num] = AL_i
out_data_dict["AL_dict"] = data_dict

In [ ]:
def get_gen_acquired(
    AL_i,
    id_of_most_stable
    # init_id_of_most_stable,
    ):
    out_dict = dict()
    out_dict["first_acq_id"] = None

    color_progression_i = AL_i.color_dict_progression.get(id_of_most_stable, None)

    gen_acquired = None

    if color_progression_i is not None:
        init_id_of_most_stable = color_progression_i[0]

        for gen_i, AL_gen_i in AL_i.al_gen_dict.items():
            # print("init_id_of_most_stable:", init_id_of_most_stable)

            acquired_i = AL_gen_i.model.loc[init_id_of_most_stable].acquired


            if acquired_i:
                print("init_id_of_most_stable:", init_id_of_most_stable)
                out_dict["first_acq_id"] = init_id_of_most_stable
                gen_acquired = gen_i
                break

    elif color_progression_i is None:
        # gen_acquired = 69

        AL_gen_i = AL_i.al_gen_dict[
            list(AL_i.al_gen_dict.keys())[-1]
            ]
        model = AL_gen_i.model
        
        out_dict["first_acq_id"] = id_of_most_stable

        gen_acquired = model.loc[id_of_most_stable].gen_acquired

    print("Acquired:", gen_acquired)

    out_dict["gen_acq"] = gen_acquired

    return(out_dict)

In [ ]:
AL_gen_i = AL_i.al_gen_dict[
    list(AL_i.al_gen_dict.keys())[-1]
    ]

model = AL_gen_i.model

In [ ]:
AL_dict = out_data_dict["AL_dict"]

runs_list = list(AL_dict.keys())

data_list = []
gen_acquired_list = []
for run_i in runs_list:
    print("run_i:", run_i)

    data_row_i = dict()

    AL_i = AL_dict[run_i]

    last_gen = list(AL_i.al_gen_dict.keys())[-1]

    tmp = AL_i.al_gen_dict[last_gen]
    duplicates = tmp.indices_that_are_duplicates

    not_dupl_list = []
    for index in tmp.model.index:
        if index not in duplicates:
            not_dupl_list.append(index)

    id_of_most_stable = tmp.model.loc[not_dupl_list].sort_values("y").iloc[0].name
    print("id_of_most_stable:", id_of_most_stable)

    # gen_acq = get_gen_acquired(
    out_dict = get_gen_acquired(
        AL_i,
        id_of_most_stable,
        )
    gen_acq = out_dict["gen_acq"]
    gen_acquired_list.append(gen_acq)

    data_list.append(out_dict)

    print("")

In [ ]:
print("Average generatios to acquire structure", "\n", np.mean(gen_acquired_list))

In [ ]:
df = pd.DataFrame(data_list)



# df.first_acq_id.value_counts()

In [ ]:
for i in df.first_acq_id.unique().tolist():
    print(i)
    
    from IPython.display import display
    df_i = df[df.first_acq_id == i]
    # display(df_i)
    
    mean_gen_acq = df_i.gen_acq.mean()
    print("mean_gen_acq:", mean_gen_acq)
    print("")

In [ ]:
assert False

In [ ]:
AL_i

# init_id_of_most_stable = 
# AL_i.color_dict_progression.get(id_of_most_stable, None)

In [ ]:
# init_id_of_most_stable

In [ ]:
id_of_most_stable

In [ ]:
run_i

In [ ]:
df_i = AL_i.al_gen_dict[48].model.sort_values("y")

df_i[df_i.duplicate == False]

In [ ]:
AL_i.color_dict_progression

# id_of_most_stable

In [ ]:
# # key_i = list(AL_dict.keys())[0]
# # AL_i = AL_dict[key_i]

# AL_i = AL_dict[run_i]

# last_gen = list(AL_i.al_gen_dict.keys())[-1]

# tmp = AL_i.al_gen_dict[last_gen]
# # tmp.model.sort_values("y")
# duplicates = tmp.indices_that_are_duplicates

# not_dupl_list = []
# for index in tmp.model.index:
#     if index not in duplicates:
#         not_dupl_list.append(index)

# id_of_most_stable = tmp.model.loc[not_dupl_list].sort_values("y").iloc[0].name

# init_id_of_most_stable = AL_i.color_dict_progression[id_of_most_stable][0]

In [ ]:
# get_gen_acquired(
#     AL_i,
#     init_id_of_most_stable,
#     )